In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

import os

In [3]:
train = pd.read_csv('./siim-isic-melanoma-classification/train.csv')
test  = pd.read_csv('./siim-isic-melanoma-classification/test.csv')
sub   = pd.read_csv('./siim-isic-melanoma-classification/sample_submission.csv')

train.shape, test.shape, sub.shape

((33126, 8), (10982, 5), (10982, 2))

In [4]:
train['sex'] = train['sex'].fillna('na')
train['age_approx'] = train['age_approx'].fillna(0)
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].fillna('na')

test['sex'] = test['sex'].fillna('na')
test['age_approx'] = test['age_approx'].fillna(0)
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].fillna('na')

In [5]:
train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0


In [6]:
test.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,male,70.0,na
1,ISIC_0052349,IP_7782715,male,40.0,lower extremity
2,ISIC_0058510,IP_7960270,female,55.0,torso
3,ISIC_0073313,IP_6375035,female,50.0,torso
4,ISIC_0073502,IP_0589375,female,45.0,lower extremity


In [10]:
L = 15
feat = ['sex','age_approx','anatom_site_general_challenge']

M = train.target.mean()

te = train.groupby(feat)['target'].agg(['mean','count']).reset_index()

te.tail(20)

,sex,age_approx,anatom_site_general_challenge,mean,count
199,male,75.0,upper extremity,0.076923,78
200,male,80.0,head/neck,0.272727,33
201,male,80.0,lower extremity,0.000000,69
202,male,80.0,na,1.000000,2
203,male,80.0,palms/soles,0.000000,1
204,male,80.0,torso,0.078740,127
205,male,80.0,upper extremity,0.108108,37
206,male,85.0,head/neck,0.071429,14
207,male,85.0,lower extremity,0.076923,13
208,male,85.0,palms/soles,1.000000,1


In [ ]:
te['ll'] = ((te['mean']*te['count'])+(M*L))/(te['count']+L)

del te['mean'], te['count']

test = test.merge( te, on=feat, how='left' )

test['ll'] = test['ll'].fillna(M)

test.head()

In [ ]:
sub.target = test.ll.values
sub.head(10)

In [ ]:
sub.to_csv( './Ensembles/baseline-data-submission.csv', index=False )